---

# **title: '머신러닝5 트리의 앙상블'**

# **date: '2022-07-05 09:00'**

---

##  랜덤 포레스트
- Decision Tree(나무 1개)에서 출발
  + 여러개 심음
  + 샘플링
  + Feature Importances
- 예측해야 할 행의 갯수,100만개
- 컬럼의 갯수 200개 ==>100개 축소
  + 나무 100개를 심고 평균을 내자
  + 나무 1개당 컬럼을 10개로 다양한 값 찾기
     + T1 mae :20, T2 mae :30, T3 mae 10.....->T1~T100 mae :20(평균값)
     + Feature Importances
  + 샘플링 : 부트스트랩 샘플(복원추출)
  

In [ ]:
# 라이브러리 불러오기 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier

# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# input, target 분리 
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()


# 훈련데이터, 테스트 데이터 분리
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42
)

# 모델링
rf = RandomForestClassifier(n_jobs=-1, random_state = 42)

# 모형 평가
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs =-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']),"과대적합")

# 특성 중요도
rf.fit(train_input, train_target)
print(rf.feature_importances_,"역시 당도가 중요")

# OOB 
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_,"OOB검증세트와 비슷")

0.9973541965122431 0.8905151032797809 과대적합
[0.23167441 0.50039841 0.26792718] 역시 당도가 중요
0.8934000384837406 OOB검증세트와 비슷


## 그레이디언트 부스팅
- 기존알고리즘에 가중치(보정치)를 주어 학습을 시킴
- 경사하강법의 원리를 이용함
- T1~Tn 증가하면서 오차를 보정해주며 정확성을 높임
- 랜덤포레스트와의 차이점
  + 랜덤포레스트는 각 나무간의 상호 연관성이 없음(부트스트랩샘플)
  + 그레이디언트 부스팅은 각 나무간 상호 연관성이 있음
     ->그러나 너무 느린 속도
  + XGBoost,LightGBM이 대안

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb,train_input,train_target, return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
#결정트리 갯수를 기본100->500개로 늘리고 학습율 기본0.1->0.2로 늘려 봄
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores = cross_validate(gb,train_input,train_target, return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


- 특성 중요도

In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]
